1.summary 

This data is coming for music company Sparkify which contain churn customer and we want to find them out

- data sourece is “https://udacity-dsnd.s3.amazonaws.com/sparkify_event_data.json”
- https://bucket-emr-15.s3.amazonaws.com/sparkify_event_data.json

In [1]:
#install and import libraries, it seems 
#it seems that we should install these packages every time before we run our Online EMR code
#session it self is depend on YARN, which means every time i start a new notebook, it will start a new session from basic virtal env

sc.install_pypi_package("pandas==1.0.3")
sc.install_pypi_package("matplotlib", "https://pypi.org/simple")
sc.install_pypi_package("scikit-learn")
sc.install_pypi_package("s3fs")
sc.list_packages()


VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1589036126722_0002,pyspark,idle,,,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached pandas-1.0.3-cp36-cp36m-manylinux1_x86_64.whl (10.0 MB)
  Using cached python_dateutil-2.8.1-py2.py3-none-any.whl (227 kB)

  Using cached matplotlib-3.2.1-cp36-cp36m-manylinux1_x86_64.whl (12.4 MB)
  Using cached pyparsing-2.4.7-py2.py3-none-any.whl (67 kB)
  Using cached kiwisolver-1.2.0-cp36-cp36m-manylinux1_x86_64.whl (88 kB)
  Using cached cycler-0.10.0-py2.py3-none-any.whl (6.5 kB)

  Using cached scikit_learn-0.22.2.post1-cp36-cp36m-manylinux1_x86_64.whl (7.1 MB)
  Using cached scipy-1.4.1-cp36-cp36m-manylinux1_x86_64.whl (26.1 MB)
  Using cached joblib-0.14.1-py2.py3-none-any.whl (294 kB)

  Using cached s3fs-0.4.2-py3-none-any.whl (19 kB)
  Using cached botocore-1.16.6-py2.py3-none-any.whl (6.2 MB)
  Using cached fsspec-0.7.3-py3-none-any.whl (70 kB)
  Using cached docutils-0.15.2-py3-none-any.whl (547 kB)
  Using cached urllib3-1.25.9-py2.py3-none-any.whl (126 kB)

Package                    Version
-------------------------- ------------
beautifulsoup4        

In [2]:
import numpy as np
import pandas as pd
import time
#this package support pandas to read files on S3 storage
import s3fs

import matplotlib.pyplot as plt

from pyspark.sql import SparkSession, Window
from pyspark.sql import functions as F
from pyspark.sql.functions import udf, lit, struct, countDistinct, collect_list, avg, count, col
from pyspark.sql.functions import sum as Fsum
from pyspark.sql.types import ArrayType, BooleanType, LongType, FloatType, IntegerType
from pyspark.ml.feature import VectorAssembler, Normalizer, StandardScaler
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, GBTClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_curve


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
spark = SparkSession \
        .builder \
        .appName("Sparkify_ML") \
        .getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
def c4t12_print(start, end):
    return print('4 Core and 0 Tasks take {} seconds in thi model'.format(end - start))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

"""
start = time.time()
df = pd.read_json(datapath_sparkify)
end = time.time()
print("Pandas load 12 GB files use", end-start)
"""

In [5]:
datapath_sparkify = 's3://bucket-emr-15/sparkify_event_data.json'
data_min = ''
start = time.time()
df = spark.read.json(datapath_sparkify)
end = time.time()
print("4 Core and 12 Task with Spark load 12 GB files use", end-start)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

4 Core and 12 Task with Spark load 12 GB files use 37.03688979148865

In [6]:
df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+---------+---------+------+-------------+--------+----------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+-------+
|              artist|     auth|firstName|gender|itemInSession|lastName|    length|level|            location|method|    page| registration|sessionId|                song|status|           ts|           userAgent| userId|
+--------------------+---------+---------+------+-------------+--------+----------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+-------+
|           Popol Vuh|Logged In|    Shlok|     M|          278| Johnson| 524.32934| paid|Dallas-Fort Worth...|   PUT|NextSong|1533734541000|    22683|Ich mache einen S...|   200|1538352001000|"Mozilla/5.0 (Win...|1749042|
|         Los Bunkers|Logged In|  Vianney|     F|            9|  Miller| 238.39302| paid|San Francisco-Oak...|  

In [9]:
df.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName', 'length', 'level', 'location', 'method', 'page', 'registration', 'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId']

In [12]:
#df.limit(5).toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                                              artist  ...   userId
0                                          Popol Vuh  ...  1749042
1                                        Los Bunkers  ...  1563081
2                                               Lush  ...  1697168
3  Barry Tuckwell/Academy of St Martin-in-the-Fie...  ...  1222580
4                                                Yes  ...  1714398

[5 rows x 18 columns]

In [7]:
print((df.count(), len(df.columns)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(26259199, 18)

In [8]:
start = time.time()
#clearning the dataset
#drop some irrelevant columns
df = df.drop(*['artist', 'song','firstName','lastName','id_copy'])
#drop some potential NAN values
df = df.dropna(how='any', subset = ['userId','sessionId'])
#filtering out the invalid Ids
df = df.filter(df.userId!='').orderBy(['userId', 'ts'], ascending=[True, True])
#case userID column to integer
df = df.withColumn("userId", df["userId"].cast(IntegerType()))



#defining the chrun
#these are customers landing on the Cancellation Confirmation page
cancelation = udf(lambda x: 1 if x == "Cancellation Confirmation" else 0, IntegerType())  
df = df.withColumn("churn", cancelation("page"))
window = Window.partitionBy("userId").rangeBetween(Window.unboundedPreceding, Window.unboundedFollowing)
df = df.withColumn("churn", Fsum("churn").over(window))

#some new columns were made in this step to make the data exploration easier, the Feature engineering step comes later

#making the level_shift Column
#this column tell us how many times did the customer switched from paid to free service
window_1 = Window.partitionBy().orderBy(['userId', 'ts'])
df = df.withColumn("level_shift", (df.level!=F.lag(df.level).over(window_1)) \
                   | (df.userId!=F.lag(df.userId).over(window_1)))
df = df.fillna({'level_shift':0})
df = df.withColumn("level_shift", F.when(df["level_shift"]==False, 0).otherwise(1))

#making the last_ts column
#this column will help us to select only records that happened in the last 2 weeks of customer activity
#the idea is that customer behavior should be different shortly before the churn happened
#trimming the last three zeros from the UNIT time(miliseconds)
df = df.withColumn("ts", df.ts/1000)
df = df.withColumn("registration", df.registration/1000)
#window_2
window_2 = Window.partitionBy("userId")
#
df = df.withColumn("last_ts", F.max("ts").over(window_2))
#2592000 this approxmiate number of seconds in a week period
df = df.filter(df.last_ts - df.ts < 1300000)

#making the columns:papes_per _seesion, diff_time
#pages_per+session is number of pages perseesion
#diff_time is a number of days since a specific page was visited

window_3 = Window.partitionBy(["userId", "sessionId"])
df = df.withColumn("pages_per_session", F.max("ItemInSession").over(window_3))
#unix to datatime
df = df.withColumn("ts_time", F.to_timestamp(df.ts))
#unix time style to normal datatime
df = df.withColumn("last_ts_time", F.to_timestamp(df.last_ts))

#how many days ago was the page visited
df = df.withColumn('diff_time', F.datediff(df.last_ts_time, df.ts_time))
df = df.orderBy(["userId", "ts"], ascending = [True, True])
#create a temp Table to be used for swl queries
df.createOrReplaceTempView('data');


end = time.time()
print("4 Core and 12 Task Cleaning Data Model take time is ", end-start)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

4 Core and 12 Task Cleaning Data Model take time is  0.8517119884490967

In [10]:
#the column "page" seems to be most informative in the whole datset

df.select('page', 'UserId').groupby('page').agg({'page':'count'}).select('page', 'count(page)').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----------+
|                page|count(page)|
+--------------------+-----------+
|              Cancel|       5003|
|    Submit Downgrade|       2450|
|         Thumbs Down|      89370|
|                Home|     425269|
|           Downgrade|      73782|
|         Roll Advert|     125746|
|              Logout|     106746|
|       Save Settings|      10755|
|Cancellation Conf...|       5003|
|               About|      27272|
| Submit Registration|         98|
|            Settings|      54420|
|               Login|      63666|
|            Register|        208|
|     Add to Playlist|     221047|
|          Add Friend|     139143|
|            NextSong|    7709280|
|           Thumbs Up|     418797|
|                Help|      52636|
|             Upgrade|      15173|
+--------------------+-----------+
only showing top 20 rows

In [11]:
df.printSchema();

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- auth: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: double (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: integer (nullable = true)
 |-- churn: long (nullable = true)
 |-- level_shift: integer (nullable = false)
 |-- last_ts: double (nullable = true)
 |-- pages_per_session: long (nullable = true)
 |-- ts_time: timestamp (nullable = true)
 |-- last_ts_time: timestamp (nullable = true)
 |-- diff_time: integer (nullable = true)

In [12]:
# Investigation if there are differences between churned and non-churned users

# label             - 0 if non_churned, 1 if churned
# song_count        - avg number of songs played by churned/non_churned users
# error             - avg number of errors occuring 
# friends           - avg number of "friends" on thhe application
# playlist_count    - avg number of visits to the Playlist page
# thumbs_up         - avg number of clicking the 'thumbs up'
# thumbs_down       - avg number of clicking the 'thumbs down'
# downgrade         - avg number of visits to the downgrade page
# count_session_dist- avg number of sessions made
# count_diff_time   - avg number of days (in the last two weeks) in which the user used the app
# pages per session - avg numbers of pages (or any activity or changes) per session
# duration          - avg number of days since the user joined (division with 86400 as a proxy of seconds within a day)
# level_shift       - avg number of level changes (free, paid) per customer
# usage_time        - avg total time spent using the app

start = time.time()
stats = spark.sql(" WITH prep as( \
SELECT userId, \
max(churn)                                                          as label, \
count(case when page = 'NextSong' then userId else null end)        as song_count, \
count(case when page = 'Error' then userId else null end)           as error, \
count(case when page = 'Add Friend' then userId else null end)      as friends, \
count(case when page = 'Add to Playlist' then userId else null end) as playlist_count, \
count(case when page = 'Thumbs Up' then userId else null end)       as thumbs_up, \
count(case when page = 'Thumbs Down' then userId else null end)     as thumbs_down, \
count(case when page = 'Downgrade' then userId else null end)       as downgrade, \
count(distinct sessionId)                                           as count_session_dist, \
count(distinct diff_time)                                           as count_diff_time, \
avg(distinct pages_per_session)                                     as pages_per_session, \
(max(ts) - min(registration))/86400                                 as duration, \
sum(level_shift)                                                    as level_shift, \
sum(length)                                                         as usage_time \
FROM data \
GROUP BY userId) \
SELECT label, \
count(label)             as cnt, \
avg(song_count)          as song_count, \
avg(error)               as error, \
avg(friends)             as friends, \
avg(playlist_count)      as playlist_count, \
avg(thumbs_up)           as thumbs_up, \
avg(thumbs_down)         as thumbs_down, \
avg(downgrade)           as downgrade, \
avg(count_session_dist)  as count_session_dist, \
avg(count_diff_time)     as count_diff_time, \
avg(pages_per_session)   as pages_per_session, \
avg(duration)            as duration, \
avg (level_shift)        as level_shift, \
avg(usage_time)          as usage_time \
FROM prep \
GROUP BY label")

# We can see that for most dimensions/features there are differences between churned and non-churned users
stats.toPandas()

end = time.time()
print('4 Core and 12 Task take {} seconds'.format(end - start))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

4 Core and 12 Task take 150.19197535514832 seconds

In [28]:
stats.show(n=2, truncate=False, vertical=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

-RECORD 0---------------------------------
 label              | 0                   
 cnt                | 17275               
 song_count         | 311.4591027496382   
 error              | 0.38309696092619394 
 friends            | 5.630448625180898   
 playlist_count     | 8.965962373371925   
 thumbs_up          | 17.397510853835023  
 thumbs_down        | 3.4504196816208395  
 downgrade          | 2.8214182344428367  
 count_session_dist | 6.465354558610709   
 count_diff_time    | 4.402257597684515   
 pages_per_session  | 88.69388326056588   
 duration           | 89.08563652583196   
 level_shift        | 2.722026049204052   
 usage_time         | 77847.83898664259   
-RECORD 1---------------------------------
 label              | 1                   
 cnt                | 5003                
 song_count         | 465.4855086947831   
 error              | 0.5628622826304217  
 friends            | 8.370377773336      
 playlist_count     | 13.224065560663602  
 thumbs_up 

In [29]:
df_5 = stats.toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
start=time.time()
# Feature Enigneering

# Based on the previous analysis, all 14 investigated features will be included. 
# Here I am making a Temp Table which holds all the features.
# The temp table will be used as model input
# All data is aggregated per userId


features = spark.sql("SELECT userId, \
max(churn)                                                          as label, \
count(case when page = 'NextSong' then userId else null end)        as song_count, \
count(case when page = 'Error' then userId else null end)           as error, \
count(case when page = 'Add Friend' then userId else null end)      as friends, \
count(case when page = 'Add to Playlist' then userId else null end) as playlist_count, \
count(case when page = 'Thumbs Up' then userId else null end)       as thumbs_up, \
count(case when page = 'Thumbs Down' then userId else null end)     as thumbs_down, \
count(case when page = 'Downgrade' then userId else null end)       as downgrade, \
count(distinct sessionId)                                           as count_session_dist, \
count(distinct diff_time)                                           as count_diff_time, \
round(avg(distinct pages_per_session),0)                                     as pages_per_session, \
round((max(ts) - min(registration))/86400,0)                                as duration, \
round(sum(level_shift),0)                                                  as level_shift, \
round(sum(length),0)                                                         as usage_time \
FROM data \
GROUP BY userId");

features.createOrReplaceTempView('features');
features=features.na.drop() # clean potential NaNs


end = time.time()
c4t12_print(start, end)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

4 Core and 12 Tasks take 0.10963582992553711 seconds in thi model

In [15]:
start=time.time()
features.show(n=2, truncate=False, vertical=True)
end = time.time()
c4t12_print(start, end)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

-RECORD 0----------------------
 userId             | 1041322  
 label              | 1        
 song_count         | 765      
 error              | 1        
 friends            | 10       
 playlist_count     | 20       
 thumbs_up          | 31       
 thumbs_down        | 7        
 downgrade          | 10       
 count_session_dist | 7        
 count_diff_time    | 9        
 pages_per_session  | 133.0    
 duration           | 108.0    
 level_shift        | 0        
 usage_time         | 190753.0 
-RECORD 1----------------------
 userId             | 1226805  
 label              | 0        
 song_count         | 756      
 error              | 2        
 friends            | 11       
 playlist_count     | 28       
 thumbs_up          | 73       
 thumbs_down        | 8        
 downgrade          | 4        
 count_session_dist | 8        
 count_diff_time    | 8        
 pages_per_session  | 117.0    
 duration           | 70.0     
 level_shift        | 2        
 usage_t

In [35]:
type(features)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'pyspark.sql.dataframe.DataFrame'>

In [36]:
df_6 = features.toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
type(df_6)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'pandas.core.frame.DataFrame'>

In [39]:
df_6.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Index(['userId', 'label', 'song_count', 'error', 'friends', 'playlist_count',
       'thumbs_up', 'thumbs_down', 'downgrade', 'count_session_dist',
       'count_diff_time', 'pages_per_session', 'duration', 'level_shift',
       'usage_time'],
      dtype='object')

In [41]:
df_6.shape

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(22192, 15)

In [44]:
df_6[df_6['label']==1]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

        userId  label  song_count  ...  duration  level_shift  usage_time
0      1065787      1         402  ...      25.0            1    100819.0
6      1711766      1        1823  ...      87.0            0    452765.0
16     1105036      1        1574  ...      65.0            1    393949.0
17     1261561      1         405  ...       9.0            2     99607.0
36     1702129      1        1427  ...      53.0            2    352510.0
...        ...    ...         ...  ...       ...          ...         ...
22161  1369625      1          93  ...      71.0            0     23665.0
22165  1505317      1         538  ...      60.0            0    132080.0
22173  1396828      1         852  ...     251.0            0    213630.0
22181  1386738      1          57  ...      71.0            0     14047.0
22190  1758708      1         350  ...      34.0            0     85104.0

[5000 rows x 15 columns]

In [ ]:
df_6[df_]

In [ ]:
fig, ax = plt.subplots()
ax.barh(group_names, group_data)

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(5, 5))
axs[0, 0].hist(df_6)
#axs[1, 0].scatter(data[0], data[1])
#axs[0, 1].plot(data[0], data[1])
#axs[1, 1].hist2d(data[0], data[1])

<br>

<br>

<br>

<br>

Models

In [16]:
start = time.time()
# Train Test Split
training, test = features.randomSplit([0.8, 0.2])


# Make VectorAssembler - this is a Pypark specific step
# All input features must be in one column before feeding into the model

assembler = VectorAssembler(inputCols=["userId","song_count","error","friends","playlist_count", \
                                       "thumbs_up","thumbs_down","downgrade", "count_session_dist",\
                                       "count_diff_time","pages_per_session", "duration","level_shift",\
                                       "usage_time"], \
                            outputCol="inputFeatures")



# Normalize Data
scaler = Normalizer(inputCol="inputFeatures", outputCol="features")


# Spark supports most common classification methods (https://spark.apache.org/docs/latest/ml-classification-regression.html)
# I decided for the following three:

lr=LogisticRegression()
gbt = GBTClassifier()
rf= RandomForestClassifier()


# Building pipelines
pipeline1=Pipeline(stages=[assembler, scaler, lr])
pipeline2=Pipeline(stages=[assembler, scaler, gbt])
pipeline3=Pipeline(stages=[assembler, scaler, rf])

end = time.time()
c4t12_print(start, end)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

4 Core and 12 Tasks take 0.10284852981567383 seconds in thi model

<br>

<br>

<br>

Logistic Regression

In [17]:
start = time.time()
# metric chosen is f1 (we want to catch true positives (churn customers), but
# we do not want to waste money on false positives (investing in retaining non-churn customers, which are loyal anyway)
# Note that Recall might also be justified to use here (if the cost of false positives is low)


paramgrid =ParamGridBuilder()\
.addGrid(lr.regParam, [0.0, 0,1])\
.addGrid(lr.maxIter, [10])\
.build()


evaluator=MulticlassClassificationEvaluator(metricName="f1")
#evaluator=BinaryClassificationEvaluator( metricName="areaUnderROC")

# Crossvalidator (https://spark.apache.org/docs/2.1.3/api/python/_modules/pyspark/ml/tuning.html)

crossval= CrossValidator(estimator=pipeline1,  
                         estimatorParamMaps=paramgrid,
                         evaluator = evaluator , 
                         numFolds=3
                        )

cvModel1=crossval.fit(training) 
evaluator.evaluate(cvModel1.transform(test))
#cvModel1.subModels() 
#cvModel1.avgMetrics

end = time.time()
c4t12_print(start, end)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

4 Core and 12 Tasks take 1062.1369140148163 seconds in thi model

Gradient Boosted Tree Classifier

In [18]:
# Specify multiple parameters in the paramgrid, in case you have enough processing power 
start = time.time()
paramgrid1 =ParamGridBuilder()\
.build()

evaluator=MulticlassClassificationEvaluator(metricName="f1")

crossval= CrossValidator(estimator=pipeline2,  
                         estimatorParamMaps=paramgrid1,
                         evaluator=evaluator, 
                         numFolds=3
                        )

cvModel2=crossval.fit(training) 
evaluator.evaluate(cvModel2.transform(test))
end = time.time()
c4t12_print(start, end)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-18:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 178, in cell_monitor
    job_binned_stages[job_id][stage_id] = all_stages[stage_id]
KeyError: 2311



4 Core and 12 Tasks take 1186.9926142692566 seconds in thi model

In [20]:
start = time.time()
# Specify multiple parameters in the paramgrid, in case you have enough processing power
paramgrid2 =ParamGridBuilder()\
.build()

evaluator=MulticlassClassificationEvaluator(metricName="f1")
#evaluator=BinaryClassificationEvaluator( metricName="areaUnderROC")

crossval= CrossValidator(estimator=pipeline3,  
                         estimatorParamMaps=paramgrid2,
                         evaluator=evaluator, 
                         numFolds=3
                        )

cvModel3=crossval.fit(training) 
evaluator.evaluate(cvModel3.transform(test))

end = time.time()
c4t12_print(start, end)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-20:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 178, in cell_monitor
    job_binned_stages[job_id][stage_id] = all_stages[stage_id]
KeyError: 6491



4 Core and 12 Tasks take 1038.940399646759 seconds in thi model

In [26]:
start = time.time()
predictions=cvModel1.transform(test)
#predictions.limit(2).toPandas()
end = time.time()
c4t12_print(start, end)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

4 Core and 12 Tasks take 0.08550763130187988 seconds in thi model

In [32]:
predictions.select['probability']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'method' object is not subscriptable
Traceback (most recent call last):
TypeError: 'method' object is not subscriptable



Visualisations

In [27]:
# Gradient Boosted Tree Classifier seems to be the winner model, let us plot its Precision-Recall Curve
 
start = time.time()
def pr(ax, predictions, labels, title='Precision-Recall curve'):
    ''' outputs the roc curve on ax axes'''
    prediction_prob = predictions.toPandas()['probability'].apply(lambda x:x[1]).values
    pr, rc, _ = precision_recall_curve(labels, prediction_prob)
    ax.plot(pr, rc)
    ax.set_facecolor('xkcd:wheat')
    ax.set_xlabel('Recall Rate')
    ax.set_ylabel('Precision Rate')
    ax.set_title(title)
   
plt.clf()

labels=predictions.toPandas()['label']
fig = plt.figure()
ax = fig.add_subplot(111)
pr(ax, predictions,labels)
%matplot plt


end = time.time()
c4t12_print(start, end)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

4 Core and 12 Tasks take 141.40236735343933 seconds in thi model

In [28]:
# Gradient Boosted Tree Classifier seems to be the winner model, let us plot its ROC Curve
 
start = time.time()
def roc(ax, predictions, labels, title='ROC curve'):
    ''' outputs the roc curve on ax axes'''
    prediction_prob = predictions.toPandas()['probability'].apply(lambda x:x[1]).values
    fpr, tpr, _ = roc_curve(labels, prediction_prob)
    ax.plot(fpr, tpr)
    ax.set_facecolor('xkcd:wheat')
    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')
    ax.set_title(title)
   

plt.clf()

labels=predictions.toPandas()['label']
fig = plt.figure()
ax = fig.add_subplot(111)
roc(ax, predictions,labels)
#plt.save_fig('lr_roc.png')
%matplot plt
end = time.time()
c4t12_print(start, end)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

4 Core and 12 Tasks take 13.51010251045227 seconds in thi model

question:

- 1.what's optimal combination of manager, core and task node. Accoridng to some AWS slide, they provide core:task propotion as 1:5
- 2.All task node is not working when i use EMR. They all idle statu during this caculation period.
- 3.For now i only use m5.xlarge as my EBS on aws. After some research, i believe c5.xlarge or c5.4xlarge will more fit EMR calcuation way. But when i try to use c5, it will give some s3 or ec2 permission error
- 4.According to AWS documents, the maxizme virul CPU in one EMR, for instance, m5.xlarge have 4 vCPU, so we can add 64 m5.xlarge in one EMR 256
- 5.Can i still use AWS educate to do more explore?